In [1]:
#Simple feature map to feed arrays into the classifier. 
import numpy as np
import json
import pickle

In [26]:
def word2features(sent, i):
    word = sent[i]
    features = [
        '',
        0,
        0,
        1,
        word.lower(),
        word[-3:],
        word[-2:],
        word.isupper(),
        word.istitle(),
        word.isdigit(),  
        0,
        0,
        ''  
    ]
    if i > 0:
        prev_word = sent[i-1]
        features[0] = prev_word.lower()
        features[1] = int(prev_word.istitle())
        features[2] = int(prev_word.isupper())
    if i < len(sent)-1:
        next_word = sent[i+1]
        features[-1] = next_word.lower()
        features[-2] = int(next_word.istitle())
        features[-3] = int(next_word.isupper())
    
    return np.array(features)

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [14]:
TAGS =  pickle.load(open( "tags.pickle", "rb" ))
TAGS.remove("O") 
NUM_TAGS = len(TAGS)

In [15]:
tag2id = {}
for id,label in enumerate(TAGS):
    tag2id[label] = id 

def label2id(labels):
    ret = []
    prev_label = ""
    for label in labels:
        if label == "O":
            ret.append([2*NUM_TAGS])
        elif label == prev_label:
            l =[tag2id[t]+ NUM_TAGS for t in label]
            ret.append(l)
        else:
            l =[tag2id[t] for t in label]
            ret.append(l)
        prev_label = label
    return ret 


In [16]:
def get_label(label_id):
    if label_id == (2*NUM_TAGS):
        return "O"
    elif label_id >= NUM_TAGS:
        return [TAGS[label_id-NUM_TAGS]]
    else:
        return TAGS[label_id]


In [17]:
def id2label(labels):
    ret = []
    for label in labels:
        l = [get_label(x) for x in label]
        if len(l) == 1 and l[0] == "O":
            l = "O"
        ret.append(l)
    return ret 

In [18]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
# load train data
with open('../../data/dev.json') as f:
    train_data = json.load(f)
    
# transform y_id to boolean vector 
def to_bool_vec(y_id):
    y_bool = np.zeros(2*NUM_TAGS+1)
    for id in y_id:
        for l in label:
            y_bool[id] = 1
    return y_bool
    
X_train = [sent2features(sent['sent']) for sent in train_data]
Y_train = [to_bool_vec(label2id(sent['tags'])) for sent in train_data]

print(X_train[0])
print("-----------------------")
print(Y_train[0])

In [28]:
# Train Model
crf_model = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=True)
crf_model.fit(X_train, Y_train)

TypeError: expected bytes, numpy.float64 found

In [ ]:
# # Make predictions on test data
# y_pred = crf.predict(X_test)

# # Print classification report
# labels = list(set(tag for sent in y_test for tag in sent))
# print(flat_classification_report(y_test, y_pred, labels=labels))